In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
import os

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
os.listdir('/content/drive/MyDrive/accelerometer_data')
#dataset = pd.read_csv('/content/drive/MyDrive/accelerometer_data')

['Deep accelerometer project', 'Deep accelerometer project-2.zip']

In [5]:
folder = '/content/drive/MyDrive/accelerometer_data/Deep accelerometer project'
os.listdir(folder)

['full_df.csv',
 'matrix_3d.npy',
 'matrix_SEQN.csv',
 'train_IDs.csv',
 'test_IDs.csv']

In [6]:
seqn = pd.read_csv(os.path.join(folder,'matrix_SEQN.csv')).iloc[:,0].values.astype(int)
full_df = pd.read_csv(os.path.join(folder,'full_df.csv'))
train_ids = pd.read_csv(os.path.join(folder,'train_IDs.csv'), header=None)[0].values.astype(int)
test_ids = pd.read_csv(os.path.join(folder,'test_IDs.csv'), header=None)[0].values.astype(int)
sequences = np.load(os.path.join(folder,'matrix_3d.npy'))

train_ids.shape
test_ids.shape
seqn.shape
sequences.shape
full_df.shape

(2052,)

(513,)

(7537,)

(7537, 7, 1440)

(9823, 3511)

In [7]:
train_sequences = sequences[np.nonzero(np.in1d(seqn, train_ids))[0],:,:] # index positions of seqn where the value is in train_ids
y_train = full_df.loc[full_df['SEQN'].isin(train_ids),'TARGET_BINARY'].to_numpy()

test_sequences = sequences[np.nonzero(np.in1d(seqn, test_ids))[0],:,:] # index positions of seqn where the value is in test_ids
y_test = full_df.loc[full_df['SEQN'].isin(test_ids),'TARGET_BINARY'].to_numpy()

# Baseline 1 a)

Very simple approach:
> Simple RNN that receives the sequences of accelerometer data in full (all 7 days combined) and predicts the class label at the end

## Pros
1. Easy to implement

## Cons
1. exploding gradients due to large amount of timesteps and no mitigation for it yet

In [35]:
import torch
from torch.utils.data import Dataset, DataLoader

class AccelerometerDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label


In [36]:
data = train_sequences.reshape(train_sequences.shape[0], -1)  # This will reshape data to (n_patients, 7*1440)


# Create an instance of your custom dataset
accelerometer_dataset = AccelerometerDataset(data, y_train)

# Create a DataLoader
dataloader = DataLoader(accelerometer_dataset, batch_size=32, shuffle=True)

In [40]:
import torch.nn as nn

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.rnn(x)
        x = x[:, -1, :]  # Get the last time step output
        x = self.fc(x)
        return x


In [46]:
model = SimpleRNN(input_size = 1, # nº features of each timestep, currently one
                  hidden_size = 200,
                  output_size = 1
                  )

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30
# Training loop
for epoch in range(num_epochs):
    for i, (data, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        # Assuming your RNN expects input of shape (batch_size, seq_length, input_size)
        outputs = model(data.unsqueeze(-1))  # Adds an input_size dimension
        loss = criterion(outputs.squeeze(), labels)  # Squeeze to match the labels' dimensions
        loss.backward()
        optimizer.step()

        if i % 5 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i}/{len(dataloader)}], Loss: {loss.item()}')

Epoch [1/30], Step [0/63], Loss: 0.6956416368484497
Epoch [1/30], Step [5/63], Loss: 0.6272827982902527
Epoch [1/30], Step [10/63], Loss: 0.7213566303253174
Epoch [1/30], Step [15/63], Loss: 0.6745216250419617
Epoch [1/30], Step [20/63], Loss: 0.6562986373901367
Epoch [1/30], Step [25/63], Loss: 0.6534990072250366
Epoch [1/30], Step [30/63], Loss: 0.6742986440658569
Epoch [1/30], Step [35/63], Loss: 0.7494781017303467
Epoch [1/30], Step [40/63], Loss: 0.6679868698120117
Epoch [1/30], Step [45/63], Loss: 0.6661444902420044


KeyboardInterrupt: 

It seems that the model isn't really learning. probably exploding gradients because each sequence is really a lot of timesteps (it's possible to check gradient explosion by taking a look at the gradients after each loss.backward())

# Baseline 1 b)

Very simple approach:
- (same) Simple RNN that receives the sequences of accelerometer data in full (all 7 days combined) and predicts the class label at the end
- gradient clipping + Xavier initialization + GRU or LSTM

## Pros
1. Easy to implement

## Cons
1. some exploration of the techniques above is required to see what works best

In [47]:
import torch.nn as nn
import torch.nn.init as init

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

        # Initialize weights
        self._init_weights()

    def forward(self, x):
        x, _ = self.rnn(x)
        x = x[:, -1, :]  # Get the last time step output
        x = self.fc(x)
        return x

    def _init_weights(self):
        # Apply Xavier uniform initialization to RNN weights
        for name, param in self.rnn.named_parameters():
            if 'weight_ih' in name:  # Weight matrix for input-to-hidden
                init.xavier_uniform_(param.data)
            elif 'weight_hh' in name:  # Weight matrix for hidden-to-hidden
                init.xavier_uniform_(param.data)

        # Apply Xavier uniform initialization to the weights of the fully connected layer
        init.xavier_uniform_(self.fc.weight)

        # Optionally initialize the bias terms (to 0, for example)
        if self.fc.bias is not None:
            init.zeros_(self.fc.bias)


In [ ]:
# for gradient clipping apply this during the training loop after loss.backward()
#> torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
#>  optimizer.step()
#>  ...


# Baseline 2

Change the way the input data is represented. maybe reduce the number of samples per person;